In [1]:
import pandas as pd
import numpy as np


In [2]:
idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format

In [3]:
df = pd.read_pickle('E:/Data/DataSQL/df_merge')
col = pd.read_excel('E:/Data/6.Python/VND-col.xlsx')
sql_col = pd.read_excel('E:/Data/6.Python/SQL-col.xlsx')
nganh = pd.read_excel('E:/Data/NganhICB.xlsx')
list_ticker = list(nganh['ticker'].values)

# GET SSI

In [4]:
import pandas as pd
import requests
from pandas import json_normalize
from io import BytesIO
import time
from datetime import datetime, timedelta
import json
import polars as pl

In [5]:

# API request config for SSI API endpoints
ssi_headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'X-Fiin-Key': 'KEY',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-Fiin-User-ID': 'ID',
        'X-Fiin-User-Token':'79,14,109,2,77,63,182,226,168,166,146,27,233,245,5,195,0,200,169,49,153,168,150,254,116,248,73,45,23,61,239,77,183,194,66,44,130,28,239,66,27,56,75,64,56,106,32,203,36,15,41,203,250,254,180,226,198,77,152,213,125,234,19,189,68,11,105,241,12,164,235,22,207,252,226,142,142,239,234,206,6,141,63,201,33,169,127,101,9,25,210,180,120,123,145,86,239,76,144,170,24,236,42,224,97,18,41,250,154,193,169,100,133,44,162,88,201,178,212,112,53,114,12,95,81,58,144,233,119,10,24,138,241,254,39,124,10,203,168,121,230,82,103,254,250,60,20,249,237,230,12,76,78,65,234,48,255,62,102,6,201,137,37,132,182,151,246,26,37,81,123,183,82,209,104,37,195,103,138,251,132,30,158,210,151,131,145,67,235,171,34,119,235,73,193,80,232,193,102,189,47,70,108,12,43,253,60,16,114,169,91,125,8,129,40,14,219,191,133,180,200,249,65,81,108,254,163,47,151,3,1,35,132,124,160,200,104,243,64,185,149,215,77,222,174,177,231,227,14,240,127,167,68,133,199,216',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        'X-Fiin-Seed': 'SEED',
        'sec-ch-ua-platform': 'Windows',
        'Origin': 'https://iboard.ssi.com.vn',
        'Sec-Fetch-Site': 'same-site',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://iboard.ssi.com.vn/',
        'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7'
        }

In [6]:
def organ_listing (lang='vi', headers=ssi_headers):
    """
    Return a DataFrame of all available stock symbols. Live data is retrieved from the SSI API.
    Parameters:
        lang (str): language of the data. Default is 'vi', other options are 'en'
        headers (dict): headers of the request
    """
    url = f"https://fiin-core.ssi.com.vn/Master/GetListOrganization?language={lang}"
    response = requests.request("GET", url, headers=headers)
    status = response.status_code
    if status == 200:
        data = response.json()
        # print('Total number of companies: ', data['totalCount'])
        df = pd.DataFrame(data['items'])
        return df
    else:
        print('Error in API response', response.text)
def financial_report (symbol='SSI', report_type='BalanceSheet', frequency='Quarterly', periods=200, latest_year=None, headers=ssi_headers): # Quarterly, Yearly
    """
    Return financial reports of a stock symbol by type and period.
    Args:
        symbol (:obj:`str`, required): 3 digits name of the desired stock.
        report_type (:obj:`str`, required): BalanceSheet, IncomeStatement, CashFlow
        report_range (:obj:`str`, required): Yearly or Quarterly.
    """
    symbol = symbol.upper()
    organ_code = organ_listing().query(f'ticker == @symbol')['organCode'].values[0]
    this_year = str(datetime.now().year)
    if latest_year == None:
      latest_year = this_year
    else:
      if isinstance(latest_year, int) != True:
        print('Please input latest_year as int number')
      else:
        pass
    url = f'https://fiin-fundamental.ssi.com.vn/FinancialStatement/Download{report_type}?language=vi&OrganCode={organ_code}&Skip=0&Frequency={frequency}&numberOfPeriod={periods}&latestYear={latest_year}'
    response = requests.get(url, headers=headers)
    # print(response.text)
    status = response.status_code
    if status == 200:
        df = pd.read_excel(BytesIO(response.content), skiprows=7).dropna()
        return df
    else:
        print(f'Error {status} when getting data from SSI. Details:\n {response.text}')
        return None


In [7]:
list_chitieu = ['TỔNG TÀI SẢN','TÀI SẢN NGẮN HẠN','Tiền và tương đương tiền','Giá trị thuần đầu tư ngắn hạn','Các khoản phải thu','Hàng tồn kho, ròng','TÀI SẢN DÀI HẠN','Phải thu dài hạn','Tài sản cố định','GTCL TSCĐ hữu hình','Nguyên giá TSCĐ hữu hình','Khấu hao lũy kế TSCĐ hữu hình','GTCL Tài sản thuê tài chính','Nguyên giá tài sản thuê tài chính','Khấu hao lũy kế tài sản thuê tài chính','GTCL tài sản cố định vô hình','Nguyên giá TSCĐ vô hình','Khấu hao lũy kế TSCĐ vô hình','Bất động sản đầu tư','Nguyên giá tài sản đầu tư','Khấu hao lũy kế tài sản đầu tư','Tài sản dở dang dài hạn','Đầu tư dài hạn',
                'NỢ PHẢI TRẢ','Nợ ngắn hạn','Phải trả người bán','Người mua trả tiền trước','Doanh thu chưa thực hiện ngắn hạn','Vay ngắn hạn','Nợ dài hạn','Người mua trả tiền trước dài hạn','Doanh thu chưa thực hiên','Vay dài hạn','Trái phiếu chuyển đổi','VỐN CHỦ SỞ HỮU','Vốn góp','Thặng dư vốn cổ phần','Cổ phiếu Quỹ','Lãi chưa phân phối','Lợi ích cổ đông không kiểm soát','Doanh số thuần','Lãi gộp','Thu nhập tài chính','Chi phí tài chính','Trong đó: Chi phí lãi vay','Lãi/(lỗ) từ công ty liên doanh','Chi phí bán hàng','Chi phí quản lý doanh  nghiệp','Thu nhập khác, ròng','Lãi/(lỗ) ròng trước thuế','Lãi/(lỗ) thuần sau thuế','Lợi nhuận của Cổ đông của Công ty mẹ',
                'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh','Khấu hao TSCĐ',
                # 'Chi phí dự phòng','Chi phí lãi vay','Chi phí lãi vay đã trả','Thuế thu nhập doanh nghiệp đã trả',
                'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư','Tiền mua tài sản cố định và các tài sản dài hạn khác','Tiền thu được từ thanh lý tài sản cố định','Cổ tức và tiền lãi nhận được','Lưu chuyển tiền tệ từ hoạt động tài chính','Tiền thu từ phát hành cổ phiếu và vốn góp','Chi trả cho việc mua lại, trả lại cổ phiếu','Tiền thu được các khoản đi vay','Tiển trả các khoản đi vay','Tiền thanh toán vốn gốc đi thuê tài chính','Cổ tức đã trả'
]
def add_ratios_Y(x):
    x = x[list_chitieu]
    x['bs_cash'] = x['Tiền và tương đương tiền'] + x['Giá trị thuần đầu tư ngắn hạn']
    x['bs_ar'] = x['Các khoản phải thu'] + x['Phải thu dài hạn']
    x['bs_fa'] = x['GTCL TSCĐ hữu hình'] + x['GTCL Tài sản thuê tài chính'] + x['GTCL tài sản cố định vô hình'] + x['Bất động sản đầu tư'] 
    x['bs_gross_fa'] = x['Nguyên giá TSCĐ hữu hình'] + x['Nguyên giá tài sản thuê tài chính'] + x['Nguyên giá TSCĐ vô hình'] + x['Nguyên giá tài sản đầu tư']
    x['other_asset'] = (x['TỔNG TÀI SẢN'] - x['bs_cash'] - x['bs_ar'] - x['Hàng tồn kho, ròng'] - x['bs_fa'] - x['Tài sản dở dang dài hạn']-x['Đầu tư dài hạn'])
    x['bs_cust_pre'] = x['Doanh thu chưa thực hiện ngắn hạn'] + x['Doanh thu chưa thực hiên'] + x['Người mua trả tiền trước'] + x['Người mua trả tiền trước dài hạn']
    x['debt'] = x['Vay ngắn hạn'] + x['Vay dài hạn'] + x['Trái phiếu chuyển đổi']
    x['other_lia'] = (x['NỢ PHẢI TRẢ'] - x['debt'] - x['Phải trả người bán'] - x['bs_cust_pre'])
    x['other_equity'] = (x['VỐN CHỦ SỞ HỮU'] - x['Vốn góp'] - x['Lãi chưa phân phối'] - x['Cổ phiếu Quỹ'])
    x['netdebt'] = (x['debt'] - x['bs_cash'])
    x['ca/ta'] = x['TÀI SẢN NGẮN HẠN']/(1+x['TỔNG TÀI SẢN'])
    x['de'] = x['debt']/(1+x['VỐN CHỦ SỞ HỮU'])

    x['tax_rate'] = 1-(x['Lãi/(lỗ) thuần sau thuế']/(1+x['Lãi/(lỗ) ròng trước thuế']))
    x['op'] = (x['Lãi gộp'] + x['Chi phí bán hàng'] + x['Chi phí quản lý doanh  nghiệp'])
    x['core_e'] = x['op'] * (1 - x['tax_rate'])
    x['fin_income'] = x['Thu nhập tài chính'] + (x['Chi phí tài chính'] - x['Trong đó: Chi phí lãi vay'])
    x['EBT'] = x['op'] + x['Trong đó: Chi phí lãi vay']
    
   
    x['cf_div'] = x['Cổ tức đã trả'] + x['Chi trả cho việc mua lại, trả lại cổ phiếu']
    x['cf_delta_debt'] = x['Tiền thu được các khoản đi vay'] + x['Tiển trả các khoản đi vay'] + x['Tiền thanh toán vốn gốc đi thuê tài chính']
    x['cf_dep'] = x['Khấu hao TSCĐ'] 
    # + x['Chi phí dự phòng']
    x['cf_khac'] = (x['Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh']+x['Lưu chuyển tiền tệ ròng từ hoạt động đầu tư']+x['Lưu chuyển tiền tệ từ hoạt động tài chính']) - (x['Lãi/(lỗ) thuần sau thuế']+x['cf_dep']+x['Tiền mua tài sản cố định và các tài sản dài hạn khác']+x['Tiền thu từ phát hành cổ phiếu và vốn góp']+x['cf_delta_debt']+x['cf_div'])
    
    x['operating_EBITDA'] = x['op']+x['cf_dep']
    x['EBITDA'] = (x['Lãi/(lỗ) ròng trước thuế']-x['Thu nhập khác, ròng'] + x['cf_dep'] - x['Trong đó: Chi phí lãi vay'])


    return x

col1 = ['Lãi gộp', 'op', 'EBT', 'Lãi/(lỗ) ròng trước thuế', 'Lãi/(lỗ) thuần sau thuế', 'Lợi nhuận của Cổ đông của Công ty mẹ', 'core_e','EBITDA']
col2 = ['Doanh số thuần', 'Lãi gộp', 'op', 'EBT', 'Lãi/(lỗ) ròng trước thuế', 'Lãi/(lỗ) thuần sau thuế', 'Lợi nhuận của Cổ đông của Công ty mẹ', 'core_e','EBITDA']

def margin_func(x):
    for i in col1:
        x[i + "_m"] = x[i] / (1+x['Doanh số thuần'])
    return x

def g_func(x):
    for i in col2:
        x["g_" + i] = x[i].pct_change(periods=1) 
    x['roe'] = x['Lãi/(lỗ) thuần sau thuế']/x['VỐN CHỦ SỞ HỮU'].rolling(2, min_periods=2).mean()
    x['roe_core'] = x['core_e']/x['VỐN CHỦ SỞ HỮU'].rolling(2, min_periods=2).mean()
    x['roa'] = x['Lãi/(lỗ) thuần sau thuế']/x['TỔNG TÀI SẢN'].rolling(2, min_periods=2).mean()
    return x

def get_fs_Y(ticker):
    bs = financial_report(ticker,'BalanceSheet','Yearly')
    # bs = bs.loc[:, (bs==0).mean() < .6]
    pl = financial_report(ticker,'IncomeStatement','Yearly')
    # pl = pl.loc[:, (pl==0).mean() < .6]
    cf = financial_report(ticker,'CashFlow','Yearly')
    cf = cf.rename(columns={'Unnamed: 0': 'CHỈ TIÊU'})
    cf.set_index('CHỈ TIÊU', inplace=True)
    cf2 = pd.DataFrame(index = {'Khấu hao TSCĐ':'CHỈ TIÊU'}, columns = cf.columns).fillna(0)
    if cf2.index[0] in cf.index:
        pass
    else:
        cf = pd.concat([cf,cf2],axis=0)
    cf = cf.reset_index().rename(columns={'index': 'CHỈ TIÊU'})

    fs = pd.concat([bs,pl,cf])
    fs = fs.T
    fs.columns = fs.iloc[0]
    fs = fs.iloc[1:,:]
    #delete all the row with NaN value > 40
    fs = fs.dropna(axis=0,thresh=40)
   #Dropping rows if more than half of the values are zeros 
    # fs = fs.loc[fs.isna().sum(axis=1)<50]
    fs = add_ratios_Y(fs)
    fs = margin_func(fs)
    fs = g_func(fs)
    fs['dates'] = fs.index.astype(int)
    return fs


def get_data_Y(ticker):
    x = get_fs_Y(ticker)
    x = pl.from_pandas(x)
    # y = get_mc(ticker)
    # x = pl.concat([y,x],how='align')
    # x = x.with_columns((pl.col('mc')/pl.col('Lãi/(lỗ) thuần sau thuế_4Q')).alias('P/E'))
    # x = x.with_columns((pl.col('mc')/pl.col('VỐN CHỦ SỞ HỮU')).alias('P/B'))
    # x = x.with_columns([(pl.col('dates').dt.strftime("%Y")).alias('dates')])
    return x

In [128]:
import os
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle,Font, Color, Alignment, Border, Side, PatternFill
desktop = os.path.expanduser("~/Desktop")
os.chdir(desktop)
list_row = ['Doanh số thuần',
    'Lãi gộp',
    'Chi phí bán hàng',
    'Chi phí quản lý doanh  nghiệp',
    'Thu nhập tài chính',
    'Chi phí tài chính',
    'Trong đó: Chi phí lãi vay',
    'Lãi/(lỗ) từ công ty liên doanh',
    'Thu nhập khác, ròng',
    'Lãi/(lỗ) ròng trước thuế',
    'Lãi/(lỗ) thuần sau thuế',
    'Lợi nhuận của Cổ đông của Công ty mẹ',
    'operating_EBITDA','EBITDA',
    
    'TÀI SẢN NGẮN HẠN','TÀI SẢN DÀI HẠN',
    'bs_cash','bs_ar','Hàng tồn kho, ròng',
    'bs_fa','Phải thu dài hạn','Tài sản dở dang dài hạn','Đầu tư dài hạn','other_asset',
    'TỔNG TÀI SẢN',
    
    'NỢ PHẢI TRẢ','Vay ngắn hạn','Vay dài hạn','bs_cust_pre','Phải trả người bán','other_lia',
    'VỐN CHỦ SỞ HỮU','Vốn góp','Thặng dư vốn cổ phần','Cổ phiếu Quỹ','Lãi chưa phân phối','Lợi ích cổ đông không kiểm soát','other_equity',
    

    'Lưu chuyển tiền thuần từ các hoạt động sản xuất kinh doanh','Khấu hao TSCĐ',
    'Lưu chuyển tiền tệ ròng từ hoạt động đầu tư','Tiền mua tài sản cố định và các tài sản dài hạn khác','Tiền thu được từ thanh lý tài sản cố định','Cổ tức và tiền lãi nhận được',
    'Lưu chuyển tiền tệ từ hoạt động tài chính','Cổ tức đã trả','Tiền thu từ phát hành cổ phiếu và vốn góp','Chi trả cho việc mua lại, trả lại cổ phiếu','Tiền thu được các khoản đi vay','Tiển trả các khoản đi vay','Tiền thanh toán vốn gốc đi thuê tài chính',
     
    'netdebt','ca/ta','de',
    'tax_rate','op','core_e','fin_income','EBT',   
    'cf_div','cf_delta_debt','cf_dep','cf_khac',
    ]


In [141]:
def save_excel(ticker):
    x = get_data_Y(ticker)
    x = x.to_pandas(x)
    x.to_excel(ticker+'.xlsx',index=False,sheet_name='Raw_data')
    x.set_index('dates',inplace=True)
    x = x.T
    wb = load_workbook(ticker+'.xlsx')
    ws = wb.active
    #create new sheet name 'Indicator'
    ws1 = wb.create_sheet('Indicator')
    #write x to the sheet ws1
    data = x.loc[list_row]/10**9
    for r in dataframe_to_rows(data, index=True, header=True):
        ws1.append(r)
    #formatting
    ws1.sheet_view.showGridLines = False
    header = NamedStyle(name="header")
    header.font = Font(bold=True, color="ffffff", size=12,name='Roboto')
    header.border = Border(bottom=Side(border_style="thin"))
    header.alignment = Alignment( horizontal="center", vertical="center")
    header.fill = PatternFill(fgColor= "14233c", fill_type="solid")
    header_row = ws1[1]
    for cell in header_row:
        cell.style = header
    #change width for first column and font to 'Roboto'
    ws1.column_dimensions['A'].width = 40
    #change text of first column font to 'Roboto'
    for cell in ws1['A']:
        cell.font = Font(size=10,name='Roboto')
    #change number format for all columns except row 1
    for col in ws1.iter_cols(min_row=2, max_row=ws1.max_row, min_col=2, max_col=ws1.max_column):
        for cell in col:
            cell.number_format = '#,##0.0'
            cell.font = Font(size=10,name='Roboto')
    #change font of worksheet to Arial

    wb.save(ticker+'.xlsx')

In [155]:
financial_report('HAG','BalanceSheet','Yearly').to_excel('HAG_BS.xlsx',index=False)

In [153]:
save_excel('HAG')


C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_25072\1099354818.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['bs_cash'] = x['Tiền và tương đương tiền'] + x['Giá trị thuần đầu tư ngắn hạn']
C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_25072\1099354818.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['bs_ar'] = x['Các khoản phải thu'] + x['Phải thu dài hạn']
C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_25072\1099354818.py:11: SettingWithCopyWarning: 
A value is trying to be set on 

In [22]:
list(x.index.get_level_values(0))

['TỔNG TÀI SẢN',
 'TÀI SẢN NGẮN HẠN',
 'Tiền và tương đương tiền',
 'Giá trị thuần đầu tư ngắn hạn',
 'Các khoản phải thu',
 'Hàng tồn kho, ròng',
 'TÀI SẢN DÀI HẠN',
 'Phải thu dài hạn',
 'Tài sản cố định',
 'GTCL TSCĐ hữu hình',
 'Nguyên giá TSCĐ hữu hình',
 'Khấu hao lũy kế TSCĐ hữu hình',
 'GTCL Tài sản thuê tài chính',
 'Nguyên giá tài sản thuê tài chính',
 'Khấu hao lũy kế tài sản thuê tài chính',
 'GTCL tài sản cố định vô hình',
 'Nguyên giá TSCĐ vô hình',
 'Khấu hao lũy kế TSCĐ vô hình',
 'Bất động sản đầu tư',
 'Nguyên giá tài sản đầu tư',
 'Khấu hao lũy kế tài sản đầu tư',
 'Tài sản dở dang dài hạn',
 'Đầu tư dài hạn',
 'NỢ PHẢI TRẢ',
 'Nợ ngắn hạn',
 'Phải trả người bán',
 'Người mua trả tiền trước',
 'Doanh thu chưa thực hiện ngắn hạn',
 'Vay ngắn hạn',
 'Nợ dài hạn',
 'Người mua trả tiền trước dài hạn',
 'Doanh thu chưa thực hiên',
 'Vay dài hạn',
 'Trái phiếu chuyển đổi',
 'VỐN CHỦ SỞ HỮU',
 'Vốn góp',
 'Thặng dư vốn cổ phần',
 'Cổ phiếu Quỹ',
 'Lãi chưa phân phối',
 'Lợi

In [35]:

x.loc[list_row]

dates,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Doanh số thuần,517.139,"1,588.031","1,880.744","4,365.309","3,969.125","3,150.252","4,394.478","2,771.219","3,054.331","6,252.447","6,711.469","4,841.225","5,388.200","2,075.444","3,176.646","2,097.418","5,110.782","6,442.397"
Lãi gộp,155.465,596.945,890.113,"2,006.762","2,007.661","1,423.952","1,200.566","1,196.126","1,226.993","1,854.426",737.714,"1,731.542","2,374.705",227.784,205.730,506.970,"1,173.401","1,293.455"
Thu nhập tài chính,1.689,409.346,438.619,199.382,"1,262.054","1,226.990",502.050,"1,003.103","1,477.819","1,053.657",957.632,"1,665.916","1,404.799","2,137.143","1,287.364",735.079,486.144,602.752
Chi phí tài chính,-16.018,-52.558,-95.798,-213.431,-216.599,-537.003,-677.235,-754.105,-633.832,"-1,203.668","-1,734.602","-1,697.932","-1,721.684","-1,963.934","-1,318.161","-1,090.293","-1,649.147",215.433
Trong đó: Chi phí lãi vay,-14.763,-49.801,-88.501,-207.444,-204.971,-464.849,-494.957,-591.864,-565.216,"-1,078.711","-1,628.402","-1,585.316","-1,532.928","-1,263.370","-1,253.571",-971.878,-793.177,270.599
Lãi/(lỗ) từ công ty liên doanh,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1.898,11.710,-18.434,64.840,12.562,8.767,-5.820,0.000,-11.069
Chi phí bán hàng,-9.883,-39.150,-75.252,-108.523,-97.473,-163.986,-166.819,-125.639,-82.213,-111.239,-169.706,-143.923,-192.446,-308.857,-354.584,-129.287,-251.939,-255.058
Chi phí quản lý doanh nghiệp,-14.780,-50.018,-125.209,-162.416,-190.206,-230.516,-244.770,-274.660,-239.958,-350.372,-511.271,-707.548,-989.336,-772.470,"-1,851.240",174.279,"1,349.895",-155.101
"Thu nhập khác, ròng",-2.066,5.149,-26.314,21.731,0.552,-17.543,-89.899,-46.321,24.425,-434.694,"-1,271.673",-399.476,-893.182,"-1,337.563",-329.336,-321.535,-80.135,102.464
Lãi/(lỗ) ròng trước thuế,114.407,869.714,"1,006.158","1,743.504","2,768.527","1,701.895",524.889,998.504,"1,773.234",806.212,"-1,980.196",430.145,47.697,"-2,005.334","-2,351.460",-130.607,"1,028.218","1,792.877"
